In [75]:
import os
import sys

import base64
import zlib
import bz2
import lzma
import zstd

from main import compress, decompress, Compressed

from llama_cpp import Llama

In [76]:
# choose a model

# throw metal warnings into the void
stderr_backup = sys.stderr
sys.stderr = open(os.devnull, 'w')

try: 
    llm = Llama.from_pretrained(
        repo_id="QuantFactory/SmolLM2-360M-GGUF",
        filename="*Q4_0.gguf",
        verbose=False,
        logits_all=True,
        n_gpu_layers=-1,
        n_ctx=32768,
    )
finally:
    sys.stderr.close()
    sys.stderr = stderr_backup

In [77]:
# choose input to compress
input_text = "The capital of France is Paris. The capital of Germany is Berlin. The capital of Italy is Rome. The capital of Spain is Madrid. The capital of Portugal is Lisbon. The capital of Belgium is Brussels. The capital of Netherlands is Amsterdam."

In [78]:
# compress text
compressed = compress(llm, input_text)
print(compressed.to_bytes())

b'\x00\x00\x001\x00\x00\x00U\xfd\xf62\xce\xbb%\x91W\x9eS\xa0'


In [79]:
# compare results
original_bytes = input_text.encode("utf-8")
original_size = len(original_bytes)

# calc llm compression ratio
compressed_size = len(compressed.to_bytes())
compression_ratio = original_size / compressed_size

# calc compression algorithms
gzip_size = len(zlib.compress(original_bytes, level=9))
bz2_size = len(bz2.compress(original_bytes, compresslevel=9))
lzma_size = len(lzma.compress(original_bytes, preset=9))
zstd_size = len(zstd.compress(original_bytes, 22))

print(f"Encoded: {base64.b64encode(compressed.to_bytes()).decode('ascii')}")
print("\nCompression Results:")
print(f"  Original:        {original_size:>6} bytes")
print(
    f"  LLM compressed:  {compressed_size:>6} bytes ({compression_ratio:.2f}x)"
)
print(
    f"  ZSTD (level 22):  {zstd_size:>6} bytes ({original_size / zstd_size:.2f}x)"
)
print(
    f"  GZIP (level 9):  {gzip_size:>6} bytes ({original_size / gzip_size:.2f}x)"
)
print(
    f"  BZ2 (level 9):   {bz2_size:>6} bytes ({original_size / bz2_size:.2f}x)"
)
print(
    f"  LZMA (level 9):  {lzma_size:>6} bytes ({original_size / lzma_size:.2f}x)"
)

Encoded: AAAAMQAAAFX99jLOuyWRV55ToA==

Compression Results:
  Original:           239 bytes
  LLM compressed:      19 bytes (12.58x)
  ZSTD (level 22):     132 bytes (1.81x)
  GZIP (level 9):     131 bytes (1.82x)
  BZ2 (level 9):      151 bytes (1.58x)
  LZMA (level 9):     192 bytes (1.24x)


In [80]:
# confirm decompression
to_decompress = Compressed.from_bytes(compressed.to_bytes())
decompressed = decompress(llm, to_decompress)
assert decompressed == input_text, "Decompression failed!"
print("\nDecompression successful!")

decompressed


Decompression successful!


'The capital of France is Paris. The capital of Germany is Berlin. The capital of Italy is Rome. The capital of Spain is Madrid. The capital of Portugal is Lisbon. The capital of Belgium is Brussels. The capital of Netherlands is Amsterdam.'